In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
aapl_text_direction_date.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_processedText_direction_date.csv
amzn_title_text_label.csv
lstm_amzn_fulltext/


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

In [ ]:
df_aapl = pd.read_csv('aapl_text_direction_date.csv')


In [ ]:
df_aapl

,title,text,direction,prediction_date,text_without_stopwords,processedText
0,The evidence is mounting that Apple&apos;s nex...,"september 2, 2018 admin trading ideas comments...",1,2018-09-04,september 2 2018 admin trading ideas comments ...,september 2 2018 trading ideas comments eviden...
1,MARKET SNAPSHOT: Stocks Sag With Nafta In Focu...,"by mark decambre, marketwatch , ryan vlastelic...",1,2018-09-04,mark decambre marketwatch ryan vlastelica s p ...,mark decambre marketwatch ryan vlastelica p 50...
2,Tracking David Einhorn's Portfolio - Q2 2018 U...,this article is part of a series that provides...,1,2018-09-04,article part series provides ongoing analysis ...,article part series provides ongoing analysis ...
3,Multiplying my 247% Profit in Pot Stocks,"frankly, i’ve just developed a brand-new tradi...",1,2018-09-04,frankly i ’ ve developed brand-new trading str...,i ve developed trading allowing us profit amer...
4,Mike Khouw's Apple Options Trade,"on cnbc's ""options action"" , mike khouw recomm...",1,2018-09-04,cnbc 's `` options action '' mike khouw recomm...,cnbc `` options action `` mike recommended lon...
...,...,...,...,...,...,...
9995,Advantages of the Individual Investor - Wealth...,advantages of the individual investor posted o...,-1,2018-06-07,advantages individual investor posted wealthy ...,advantages individual investor posted content ...
9996,Global Seed Processing Machine Market 2018 Per...,search for: global seed processing machine mar...,-1,2018-06-07,search for global seed processing machine mark...,search for global processing machine market 20...
9997,Dark mode looks great on the Mac — but it's iO...,advertisement business dark mode looks great o...,-1,2018-06-07,advertisement business dark mode looks great m...,advertisement business mode looks great mac — ...
9998,UBS Group Analysts Give Apple (AAPL) a $210.00...,tweet\napple (nasdaq:aapl) received a $210.00 ...,-1,2018-06-07,tweet apple nasdaq aapl received 210.00 price ...,tweet apple nasdaq aapl received 210.00 price ...


In [ ]:
df_aapl = df_aapl.dropna()
df_aapl.reset_index(drop=True, inplace=True)

In [ ]:
len(df_aapl)

9997

In [ ]:
!pip install scikeras
from sklearn.model_selection import TimeSeriesSplit, train_test_split, KFold, cross_validate, cross_val_score, cross_val_predict
from scikeras.wrappers import KerasClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def split_generator(df):
  cv=[]
  df['prediction_date'] = pd.to_datetime(df['prediction_date'], errors='coerce')
  groups = df.groupby(df['prediction_date'].dt.to_period("M")).groups
  split_groups = [value for (key, value) in groups.items()] 

  print('spilt number:' + str(len(split_groups)))
  for i in range(len(split_groups)):
    data_index=[]
    for j in range(len(split_groups)):
      if i != j:
        data_index.extend(split_groups[j].tolist())
    cv.append((data_index, split_groups[i].tolist()))
    # yield data_index, split_groups[i].tolist()
  
  return cv

In [ ]:
aapl_cv=split_generator(df_aapl)


spilt number:15


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from keras.optimizers import Adam, SGD

In [ ]:
def lstm_model(shape):
  layers = [LSTM(128,input_shape=(1,shape),dropout=0.2), Dense(2)]
  model = tf.keras.models.Sequential(layers)
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.SGD(),metrics=['accuracy'])
  keras_clf = KerasClassifier(model = model, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'], optimizer=tf.keras.optimizers.SGD(), epochs=40, batch_size=64, verbose=2,validation_split=0.2)

  #predictions = cross_val_predict(keras_clf, TrainX, trainY, cv=3)
  return keras_clf

In [ ]:
def lstm_cv(stock, desc, features, label, cv, print_summary=False, print_box_plot=False):
    print('Running: ', len(features))
    X = features
    y = label
    model = lstm_model(X.shape[1])
    print(X.shape)
    TrainX = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    
    print("model: ", model)

    results = pd.DataFrame(
        cross_validate(model, TrainX, y, cv=cv, scoring=['accuracy', 'precision', 'recall', 'f1']))

    if print_box_plot:
        results.drop('fit_time', axis=1).plot.box(figsize=(10, 5))

    summary = results.describe()
    if print_summary:
        print(summary.loc['mean', :])

    summary = summary.loc['mean', :]
    summary['stock'] = stock
    summary['desc'] = desc
    #print(summary.shape)
    return summary

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


In [ ]:

text_aapl=df_aapl['processedText'].tolist()
label_aapl=df_aapl['direction'].to_numpy()


print(text_aapl[0])
print(label_aapl[0])

september 2 2018 trading ideas comments evidence apple next big product pair computer glasses aapl apple said earlier week bought small colorado company worked augmented reality glasses it pretty clear point apple working pair course apple working new products 5 billion headquarters research labs
1


In [ ]:
class_label_aapl = (label_aapl + 1) / 2

In [ ]:

text_tokenized_aapl, text_tokenizer_aapl=tokenize(text_aapl)

text_padded_aapl=pad_sequences(text_tokenized_aapl, padding='post')

In [ ]:

print(text_padded_aapl[0])

[176  15  20 ...   0   0   0]


In [ ]:
summary_aapl = lstm_cv('AAPL', 'AAPL using news body', text_padded_aapl, class_label_aapl, aapl_cv)

Running:  9997
(9997, 10268)
model:  KerasClassifier(
	model=<keras.engine.sequential.Sequential object at 0x7facf86ee090>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=<keras.optimizers.optimizer_v2.gradient_descent.SGD object at 0x7facf875af50>
	loss=<keras.losses.SparseCategoricalCrossentropy object at 0x7facf86cbf50>
	metrics=['accuracy']
	batch_size=64
	validation_batch_size=None
	verbose=2
	callbacks=None
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=40
	class_weight=None
)


Epoch 1/40
125/125 - 9s - loss: 0.7192 - accuracy: 0.6575 - val_loss: 0.7023 - val_accuracy: 0.4069 - 9s/epoch - 72ms/step
Epoch 2/40
125/125 - 7s - loss: 0.6993 - accuracy: 0.5797 - val_loss: 0.6874 - val_accuracy: 0.8559 - 7s/epoch - 56ms/step
Epoch 3/40
125/125 - 7s - loss: 0.6950 - accuracy: 0.6854 - val_loss: 0.6897 - val_accuracy: 0.7177 - 7s/epoch - 53ms/step
Epoch 4/40
125/125 - 6s - loss: 0.6944 - accuracy: 0.5903 - val_loss: 0.6924 - val_accuracy: 0.6742 - 6s/epoch - 51ms/step
Epoch 5/40
125/125 - 8s - loss: 0.6933 - accuracy: 0.6347 - val_loss: 0.6888 - val_accuracy: 0.6396 - 8s/epoch - 60ms/step
Epoch 6/40
125/125 - 7s - loss: 0.6919 - accuracy: 0.6590 - val_loss: 0.6845 - val_accuracy: 0.9239 - 7s/epoch - 57ms/step
Epoch 7/40
125/125 - 6s - loss: 0.6916 - accuracy: 0.6492 - val_loss: 0.6926 - val_accuracy: 0.5531 - 6s/epoch - 51ms/step
Epoch 8/40
125/125 - 6s - loss: 0.6914 - accuracy: 0.7337 - val_loss: 0.6895 - val_accuracy: 0.8719 - 6s/epoch - 51ms/step
Epoch 9/40
125/1

Epoch 1/40
116/116 - 8s - loss: 0.7164 - accuracy: 0.5216 - val_loss: 0.7181 - val_accuracy: 0.9494 - 8s/epoch - 68ms/step
Epoch 2/40
116/116 - 6s - loss: 0.7067 - accuracy: 0.4805 - val_loss: 0.6972 - val_accuracy: 0.7033 - 6s/epoch - 50ms/step
Epoch 3/40
116/116 - 6s - loss: 0.6994 - accuracy: 0.5870 - val_loss: 0.6984 - val_accuracy: 0.4141 - 6s/epoch - 50ms/step
Epoch 4/40
116/116 - 6s - loss: 0.6964 - accuracy: 0.5859 - val_loss: 0.7018 - val_accuracy: 0.4658 - 6s/epoch - 51ms/step
Epoch 5/40
116/116 - 7s - loss: 0.6972 - accuracy: 0.5927 - val_loss: 0.6926 - val_accuracy: 0.8288 - 7s/epoch - 57ms/step
Epoch 6/40
116/116 - 7s - loss: 0.6948 - accuracy: 0.6129 - val_loss: 0.6933 - val_accuracy: 0.7954 - 7s/epoch - 61ms/step
Epoch 7/40
116/116 - 6s - loss: 0.6944 - accuracy: 0.5731 - val_loss: 0.6983 - val_accuracy: 0.4502 - 6s/epoch - 50ms/step
Epoch 8/40
116/116 - 6s - loss: 0.6939 - accuracy: 0.6122 - val_loss: 0.6954 - val_accuracy: 0.5875 - 6s/epoch - 50ms/step
Epoch 9/40
116/1

Epoch 1/40
117/117 - 8s - loss: 0.7211 - accuracy: 0.5132 - val_loss: 0.7023 - val_accuracy: 0.6056 - 8s/epoch - 72ms/step
Epoch 2/40
117/117 - 6s - loss: 0.6991 - accuracy: 0.7296 - val_loss: 0.6946 - val_accuracy: 0.4906 - 6s/epoch - 50ms/step
Epoch 3/40
117/117 - 6s - loss: 0.6994 - accuracy: 0.5633 - val_loss: 0.6910 - val_accuracy: 0.8418 - 6s/epoch - 51ms/step
Epoch 4/40
117/117 - 6s - loss: 0.6983 - accuracy: 0.6511 - val_loss: 0.6929 - val_accuracy: 0.6344 - 6s/epoch - 50ms/step
Epoch 5/40
117/117 - 6s - loss: 0.6961 - accuracy: 0.6639 - val_loss: 0.6970 - val_accuracy: 0.4687 - 6s/epoch - 51ms/step
Epoch 6/40
117/117 - 7s - loss: 0.6946 - accuracy: 0.6621 - val_loss: 0.6911 - val_accuracy: 0.8092 - 7s/epoch - 57ms/step
Epoch 7/40
117/117 - 7s - loss: 0.6937 - accuracy: 0.6869 - val_loss: 0.6870 - val_accuracy: 0.9070 - 7s/epoch - 61ms/step
Epoch 8/40
117/117 - 6s - loss: 0.6920 - accuracy: 0.7649 - val_loss: 0.6893 - val_accuracy: 0.8359 - 6s/epoch - 50ms/step
Epoch 9/40
117/1

Epoch 1/40
116/116 - 11s - loss: 0.7181 - accuracy: 0.5899 - val_loss: 0.7016 - val_accuracy: 0.3801 - 11s/epoch - 92ms/step
Epoch 2/40
116/116 - 6s - loss: 0.6997 - accuracy: 0.5229 - val_loss: 0.6899 - val_accuracy: 0.6334 - 6s/epoch - 50ms/step
Epoch 3/40
116/116 - 6s - loss: 0.6987 - accuracy: 0.5811 - val_loss: 0.6999 - val_accuracy: 0.4563 - 6s/epoch - 50ms/step
Epoch 4/40
116/116 - 6s - loss: 0.6987 - accuracy: 0.6516 - val_loss: 0.6926 - val_accuracy: 0.5929 - 6s/epoch - 50ms/step
Epoch 5/40
116/116 - 6s - loss: 0.6955 - accuracy: 0.5981 - val_loss: 0.6986 - val_accuracy: 0.4827 - 6s/epoch - 50ms/step
Epoch 6/40
116/116 - 6s - loss: 0.6936 - accuracy: 0.5444 - val_loss: 0.6924 - val_accuracy: 0.8045 - 6s/epoch - 50ms/step
Epoch 7/40
116/116 - 6s - loss: 0.6930 - accuracy: 0.6492 - val_loss: 0.6988 - val_accuracy: 0.4163 - 6s/epoch - 50ms/step
Epoch 8/40
116/116 - 6s - loss: 0.6964 - accuracy: 0.6309 - val_loss: 0.6995 - val_accuracy: 0.4012 - 6s/epoch - 50ms/step
Epoch 9/40
116

...

Epoch 1/40
118/118 - 8s - loss: 0.7170 - accuracy: 0.6336 - val_loss: 0.6995 - val_accuracy: 0.5833 - 8s/epoch - 71ms/step
Epoch 2/40
118/118 - 6s - loss: 0.7017 - accuracy: 0.6347 - val_loss: 0.6927 - val_accuracy: 0.7956 - 6s/epoch - 51ms/step
Epoch 3/40
118/118 - 6s - loss: 0.7019 - accuracy: 0.5589 - val_loss: 0.6933 - val_accuracy: 0.7282 - 6s/epoch - 50ms/step
Epoch 4/40
118/118 - 8s - loss: 0.6969 - accuracy: 0.6566 - val_loss: 0.6952 - val_accuracy: 0.7054 - 8s/epoch - 67ms/step
Epoch 5/40
118/118 - 6s - loss: 0.6939 - accuracy: 0.6572 - val_loss: 0.6989 - val_accuracy: 0.5265 - 6s/epoch - 51ms/step
Epoch 6/40
118/118 - 6s - loss: 0.6947 - accuracy: 0.6499 - val_loss: 0.6915 - val_accuracy: 0.7903 - 6s/epoch - 50ms/step
Epoch 7/40
118/118 - 6s - loss: 0.6929 - accuracy: 0.7314 - val_loss: 0.6939 - val_accuracy: 0.9374 - 6s/epoch - 50ms/step
Epoch 8/40
118/118 - 6s - loss: 0.6922 - accuracy: 0.7297 - val_loss: 0.6965 - val_accuracy: 0.5350 - 6s/epoch - 50ms/step
Epoch 9/40
118/1

Epoch 1/40
115/115 - 8s - loss: 0.7162 - accuracy: 0.6087 - val_loss: 0.6992 - val_accuracy: 0.5937 - 8s/epoch - 67ms/step
Epoch 2/40
115/115 - 6s - loss: 0.6997 - accuracy: 0.4620 - val_loss: 0.6936 - val_accuracy: 0.4661 - 6s/epoch - 48ms/step
Epoch 3/40
115/115 - 6s - loss: 0.6971 - accuracy: 0.5493 - val_loss: 0.7388 - val_accuracy: 0.2010 - 6s/epoch - 49ms/step
Epoch 4/40
115/115 - 6s - loss: 0.7025 - accuracy: 0.4254 - val_loss: 0.7034 - val_accuracy: 0.3938 - 6s/epoch - 49ms/step
Epoch 5/40
115/115 - 6s - loss: 0.6952 - accuracy: 0.4980 - val_loss: 0.6970 - val_accuracy: 0.4144 - 6s/epoch - 48ms/step
Epoch 6/40
115/115 - 6s - loss: 0.6981 - accuracy: 0.5268 - val_loss: 0.6942 - val_accuracy: 0.6513 - 6s/epoch - 49ms/step
Epoch 7/40
115/115 - 6s - loss: 0.6975 - accuracy: 0.5265 - val_loss: 0.6952 - val_accuracy: 0.5209 - 6s/epoch - 48ms/step
Epoch 8/40
115/115 - 7s - loss: 0.6995 - accuracy: 0.4856 - val_loss: 0.6961 - val_accuracy: 0.6594 - 7s/epoch - 60ms/step
Epoch 9/40
115/1

Epoch 1/40
116/116 - 8s - loss: 0.7133 - accuracy: 0.5455 - val_loss: 0.6899 - val_accuracy: 0.7268 - 8s/epoch - 71ms/step
Epoch 2/40
116/116 - 6s - loss: 0.7025 - accuracy: 0.6692 - val_loss: 0.6933 - val_accuracy: 0.8179 - 6s/epoch - 49ms/step
Epoch 3/40
116/116 - 6s - loss: 0.6968 - accuracy: 0.5678 - val_loss: 0.6901 - val_accuracy: 0.6864 - 6s/epoch - 49ms/step
Epoch 4/40
116/116 - 6s - loss: 0.6981 - accuracy: 0.6297 - val_loss: 0.6996 - val_accuracy: 0.4714 - 6s/epoch - 49ms/step
Epoch 5/40
116/116 - 7s - loss: 0.6962 - accuracy: 0.4573 - val_loss: 0.6927 - val_accuracy: 0.7861 - 7s/epoch - 60ms/step
Epoch 6/40
116/116 - 6s - loss: 0.6937 - accuracy: 0.5900 - val_loss: 0.6982 - val_accuracy: 0.5081 - 6s/epoch - 55ms/step
Epoch 7/40
116/116 - 6s - loss: 0.6926 - accuracy: 0.5618 - val_loss: 0.6940 - val_accuracy: 0.5393 - 6s/epoch - 49ms/step
Epoch 8/40
116/116 - 6s - loss: 0.6919 - accuracy: 0.6114 - val_loss: 0.6908 - val_accuracy: 0.7511 - 6s/epoch - 49ms/step
Epoch 9/40
116/1

Epoch 1/40
112/112 - 9s - loss: 0.7189 - accuracy: 0.6635 - val_loss: 0.6898 - val_accuracy: 0.5862 - 9s/epoch - 80ms/step
Epoch 2/40
112/112 - 6s - loss: 0.6978 - accuracy: 0.5107 - val_loss: 0.6996 - val_accuracy: 0.4627 - 6s/epoch - 51ms/step
Epoch 3/40
112/112 - 6s - loss: 0.6950 - accuracy: 0.4501 - val_loss: 0.6933 - val_accuracy: 0.5778 - 6s/epoch - 51ms/step
Epoch 4/40
112/112 - 6s - loss: 0.6936 - accuracy: 0.5338 - val_loss: 0.7112 - val_accuracy: 0.1325 - 6s/epoch - 50ms/step
Epoch 5/40
112/112 - 6s - loss: 0.6919 - accuracy: 0.4855 - val_loss: 0.7139 - val_accuracy: 0.1595 - 6s/epoch - 50ms/step
Epoch 6/40
112/112 - 6s - loss: 0.6915 - accuracy: 0.5422 - val_loss: 0.6975 - val_accuracy: 0.5660 - 6s/epoch - 50ms/step
Epoch 7/40
112/112 - 8s - loss: 0.6938 - accuracy: 0.5061 - val_loss: 0.6903 - val_accuracy: 0.5935 - 8s/epoch - 68ms/step
Epoch 8/40
112/112 - 6s - loss: 0.6941 - accuracy: 0.5687 - val_loss: 0.7058 - val_accuracy: 0.2993 - 6s/epoch - 51ms/step
Epoch 9/40
112/1

Epoch 1/40
125/125 - 9s - loss: 0.7149 - accuracy: 0.6003 - val_loss: 0.6941 - val_accuracy: 0.4694 - 9s/epoch - 74ms/step
Epoch 2/40
125/125 - 6s - loss: 0.7004 - accuracy: 0.7134 - val_loss: 0.6918 - val_accuracy: 0.7184 - 6s/epoch - 49ms/step
Epoch 3/40
125/125 - 6s - loss: 0.6972 - accuracy: 0.5842 - val_loss: 0.6892 - val_accuracy: 0.8102 - 6s/epoch - 49ms/step
Epoch 4/40
125/125 - 6s - loss: 0.6952 - accuracy: 0.6235 - val_loss: 0.6867 - val_accuracy: 0.8429 - 6s/epoch - 49ms/step
Epoch 5/40
125/125 - 6s - loss: 0.6916 - accuracy: 0.6778 - val_loss: 0.6935 - val_accuracy: 0.5884 - 6s/epoch - 49ms/step
Epoch 6/40
125/125 - 6s - loss: 0.6931 - accuracy: 0.6411 - val_loss: 0.6868 - val_accuracy: 0.8183 - 6s/epoch - 49ms/step
Epoch 7/40
125/125 - 6s - loss: 0.6940 - accuracy: 0.6806 - val_loss: 0.6993 - val_accuracy: 0.4794 - 6s/epoch - 49ms/step
Epoch 8/40
125/125 - 8s - loss: 0.6913 - accuracy: 0.5756 - val_loss: 0.6890 - val_accuracy: 0.7349 - 8s/epoch - 65ms/step
Epoch 9/40
125/1

In [ ]:
summary_aapl

fit_time                    258.959293
score_time                    0.771393
test_accuracy                 0.458037
test_precision                0.489954
test_recall                   0.326497
test_f1                       0.314055
stock                             AAPL
desc              AAPL using news body
Name: mean, dtype: object

In [ ]:
!pwd

/content/drive/Shareddrives/CSE573/preprocessed


In [ ]:
summary_aapl.to_csv('text_kfold_aapl.csv')